#### Introduction
March 2020 is definitely unprecedented in the history of financial markets. Both market volume and volatility heightened during this 2020 stock market crash. The market wide circuit breaker was triggered four times in 10 days (Mar 9, 12, 16 and 18) and thousands of individual stocks had circuit breaker activated. There is no respite in sight for this volatility.

The Limit Up Limit Down (LULD) was designed to replace the single stock circuit breaker. A stock enters a Limit State if its price deviation from previous close exceeds a certain percentage, which depends on the price of the stock. For Tier 1 securities, the threshold percentage from market open auction to 3:35pm is as follows: https://www.nasdaqtrader.com/content/MarketRegulation/LULD_FAQ.pdf

#### Write a q function calculateLULD
* calculate the LULD prices for a single price or list of prices
* function returns a list (upperBound, lowerBound)

In [ ]:
/ The Selector function splits the incoming data
/ calls the function that is associated with each tier

calculateLULD:{
    $[
        x>3.0;upperLimit[x];
        x<0.75;lowerLimit[x];
        middleLimit[x]
    ]
 }

In [ ]:
/ The problem with this implementation is that
/ 1.05*x changes the inherent value of x

upperLimit:{
    [x] 
    bounds:enlist(roundOff a:1.05*x, roundOff b:0.9047*x)
    
 }

In [ ]:
middleLimit:{
    [x]
    bounds:enlist(roundOff a:1.2*x, roundOff b:0.6666*x)
 }

In [ ]:
lowerLimit:{
    [x]
    result:0.75*x;
    $[
        result>0.15;
        enlist 0.15;
        enlist result
    ]
 }

In [ ]:
roundOff:{
    0.01*floor 0.5+x*100
 }

In [ ]:
sampleData:( 0.1 0.5 3 10 )
Result:enlist(calculateLULD '[sampleData])
Result

In [ ]:
/ the formating of Result 
/ doesnt seem right but calling type, shows that Result is a list

type(Result)

### Suggested Solution


In [ ]:
calculateLUID:{
    / Make Sure Input is a list of float
    price:"f" $(),x;
    
    / Defines percentage bands
    bandPctCutoff:0.75 0.2 0.05;
    
    / Defines the lower bound price of each price
    lowerBound:0.15 0w 0w;

    / Finds the percentage band bucket which each price falls into
    / Note that 0.74999 is used, instead of 0.75
    loc:0.74999 3f binr price;

    / Finds the price bands for the given list of prices
    priceBand:lowerBound[loc]&price*bandPctCutoff loc;
    
    / Applies the price bands to the given list of prices
    flip(price+priceBand;price-priceBand)
 }

**So how does it work**

In [ ]:
loc:0.74999 3f binr 0.1 0.5 3 10
lowerBound:0.15 0w 0w;
bandPctCutoff:0.75 0.2 0.05;
l:lowerBound[loc]&0.1 0.5 3 10*bandPctCutoff[loc];

In [ ]:
/ As we can see the first set is to get the class of each 
/ respective value binr returns the index of the lhs here rhs >=
loc

In [ ]:
/ This finds the lower bound
lowerBound[loc]

In [ ]:
/ This finds the upper bound
0.1 0.5 3 10*bandPctCutoff[loc]

In [ ]:
/ & Operator will return the smaller of (Left list) (Right list)
/ element wise
show priceBand:lowerBound[loc]&0.1 0.5 3 10*bandPctCutoff[loc];

In [ ]:
flip(price+priceBand;price-priceBand)

In [ ]:
price:0.1 0.5 3 10
type(price+priceBand;price-priceBand)
type(flip(price+priceBand;price-priceBand))

**Alternative Solution**

In [ ]:
calculateLULD:{
    d:`s#0 .2 .75 3.0001!(.75;0n;.2;.05);
    p:.15^x*d "f"$(),x;
    flip (x+p;x-p)
  };

In [ ]:
calculateLUID 0.1 0.5 3 10

`

when the dictionary has sorted attribute it uses bin (https://code.kx.com/q/ref/bin/) to look up keys instead of regular linear search. The bin function gets the first index that is <=val, in this case the first key that is <= lookup value. If you lookup the key .8, bin will return the index for .75 and index that into the key list, which it then looks up to get the value for the .75 key. This is also known as a step function so in the above, d "f"(),x will return you a list of values (in the domain of (.75;0n;.2;.05)) and the null values get filled with .15. Null is returned only if the price is between .2<=x<.75, because the price bands in that range are always < .75% of price dif

`